#  Performance across days

In [ ]:
# imports
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
# import holoviews.plotting.mpl
from bokeh.resources import INLINE

import paths
import functions_bondjango as bd
import pandas as pd
import numpy as np
import functions_plotting as fp
import functions_data_handling as fd
import functions_io as fi
from scipy.stats import sem
# define the name to be used for the saved figures
save_name = 'acrossDays'
line_width = 5

In [ ]:
# load the data
# get the data paths
try:
    data_path = snakemake.input[0]
except NameError:
    # define the search string
    search_string = 'result:ALL, lighting:normal, rig:miniscope, =analysis_type:aggFull'
    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = data_all[0]['analysis_path']
print(data_path)

# load the data
data = fd.aggregate_loader(data_path)

In [ ]:
# group the data

grouped_data = data.groupby(['trial_id']).agg(list)
print(grouped_data.shape)
print(grouped_data.columns)

In [ ]:
# parse the paths

# print(data_all[0])
# get the input paths
input_paths = data_all[0]['input_path'].split(',')
# allocate the counter
performance_counter = 0;
# allocate a dict for the paths
input_dict = {}

# parse the paths
input_list = [fi.parse_path(el) for el in input_paths]

# run through the paths
for idx, item in enumerate(input_list):

    # add the information to the dataframe
    grouped_data.loc[idx, 'animal'] = item['animal']
    grouped_data.loc[idx, 'date'] = item['datetime'].date()
    grouped_data.loc[idx, 'time'] = item['datetime'].time()
    grouped_data.loc[idx, 'result'] = 1 if item['result']=='succ' else 0

In [ ]:
# Get the performance

# group by the result and date
performance_info = grouped_data[['result', 'animal','date']].groupby(['animal', 'date']).agg(list)

# allocate a dict to store the performances per date per mouse
performance_dict = {}
# for all the items
for key, value in performance_info['result'].items():
    # get the current mouse
    mouse = key[0]
    # if it's not a new key
    if mouse in performance_dict.keys():
        # add the entry to the dict
        performance_dict[mouse].append([key[1], np.mean(value)])
    else:
        performance_dict[mouse] = [[key[1], np.mean(value)]]

# plot the performance

# allocate memory for the curves
plot_list = []
# for all the mice
for mouse in performance_dict.keys():
    
    # get the data in array form
    data_array = np.array(performance_dict[mouse])
    # reset the dates
#     print(np.array(performance_dict[mouse]))
    dates = data_array[:, 0] - data_array[0, 0]
    # convert the date to days
    dates = [el.days for el in dates]
    # get the performances
    performances = data_array[:, 1]
    
    curve = hv.Curve((dates, performances), 'Days','Performance')
    
    plot_list.append(curve)
    
overlay = hv.Overlay(plot_list)

overlay.opts(opts.Curve(width=fp.pix(4), height=fp.pix(4.4), toolbar=None, 
                        hooks=[fp.margin], fontsize=fp.font_sizes['small'], 
                        line_width=line_width, xticks=3, yticks=3, ylim=(0, 1.1)),
            opts.Overlay(legend_position='right', text_font='Arial', text_color='black'))

# assemble the save path
save_path = os.path.join(paths.figures_path,save_name+'_performance.png')
hv.save(overlay,save_path)
overlay

In [ ]:
# calculate capture length, 

# group the time vector based on trial
time_vector = grouped_data[['time_vector', 'animal', 'date', 'result']].groupby(['animal', 'date', 'result']).max()
# allocate a dict to store the performances per date per mouse
time_dict = {}
# for all the items
for key, value in time_vector['time_vector'].items():
    # if it's a fail, skip it
    if key[2] == 0:
        continue
    # get the current mouse
    mouse = key[0]
    # if it's not a new key
    if mouse in time_dict.keys():
        # add the entry to the dict
        time_dict[mouse].append([key[1], np.mean(value)])
    else:
        time_dict[mouse] = [[key[1], np.mean(value)]]

# plot the performance

# allocate memory for the curves
plot_list = []
# for all the mice
for mouse in time_dict.keys():
    
    # get the data in array form
    data_array = np.array(time_dict[mouse])
    # reset the dates
#     print(np.array(performance_dict[mouse]))
    dates = data_array[:, 0] - data_array[0, 0]
    # convert the date to days
    dates = [el.days for el in dates]
    # get the times
    times = data_array[:, 1]
    curve = hv.Curve((dates, times), 'Days','Total time (s)')
    plot_list.append(curve)
    
overlay = hv.Overlay(plot_list)
overlay.opts(opts.Curve(width=fp.pix(4), height=fp.pix(4.4), toolbar=None, 
                        hooks=[fp.margin], fontsize=fp.font_sizes['small'], line_width=line_width, xticks=3, yticks=3),
            opts.Overlay(legend_position='right', text_font='Arial'))

# assemble the save path
save_path = os.path.join(paths.figures_path,save_name+'_time.png')
hv.save(overlay,save_path)
overlay

In [ ]:
# calculate delay to onset

# group the time vector based on trial
delay_vector = grouped_data[['mouse_cricket_distance', 'time_vector', 'animal', 'date', 'result']]\
                            .groupby(['animal', 'date', 'result']).agg(list)
# print(delay_vector)
# allocate a dict to store the performances per date per mouse
delay_dict = {}
# for all the items
for key, value in delay_vector['mouse_cricket_distance'].items():
    # if it's a fail, skip
    if key[2] == 0:
        continue
    # get the current mouse
    mouse = key[0]
    # for each trial, get the delay to the first encounter
    delays = [np.argwhere(np.array(el)[10:]<100) for el in value]
    # assign nan for the empty entries
    delays = [el[0] if len(el)>0 else [300] for el in delays]
    
    # get the time vector
    time_vector = delay_vector['time_vector'][key]
    # for each one of the indexes, get the time
    delays = np.array([time_vector[idx][el[0]] if el[0] < 300 else 300 for idx, el in enumerate(delays)])
    
#     print([len(el) for el in value])
    # if it's not a new key
    if mouse in delay_dict.keys():
        # add the entry to the dict
        delay_dict[mouse].append([key[1], np.mean(delays)])
    else:
        delay_dict[mouse] = [[key[1], np.mean(delays)]]

# plot the performance
# allocate memory for the curves
plot_list = []
# for all the mice
for mouse in delay_dict.keys():
    
    # get the data in array form
    data_array = np.array(delay_dict[mouse])
    # reset the dates
#     print(np.array(performance_dict[mouse]))
    dates = data_array[:, 0] - data_array[0, 0]
    # convert the date to days
    dates = [el.days for el in dates]
    # get the times
    times = data_array[:, 1]
    curve = hv.Curve((dates, times), 'Days','Delay to onset (s)')
    plot_list.append(curve)


overlay = hv.Overlay(plot_list)
overlay.opts(opts.Curve(width=fp.pix(4), height=fp.pix(4.4), toolbar=None, 
                        hooks=[fp.margin], fontsize=fp.font_sizes['small'], line_width=line_width, xticks=3, yticks=3),
            opts.Overlay(legend_position='right', text_font='Arial'))
# assemble the save path
save_path = os.path.join(paths.figures_path,save_name+'_delay.png')
hv.save(overlay,save_path)
overlay